In [1]:
import cv2
import numpy as np
import random
import os


In [2]:
def add_gaussian_noise(image, mean=0, sigma=25):
    """Adds Gaussian noise to the image."""
    gauss = np.random.normal(mean, sigma, image.shape).astype('uint8')
    noisy = cv2.add(image, gauss)
    return noisy

In [3]:
def add_salt_and_pepper_noise(image, salt_prob=0.02, pepper_prob=0.02):
    """Adds salt-and-pepper noise to the image."""
    noisy = np.copy(image)
    total_pixels = image.size
    salt_pixels = int(salt_prob * total_pixels)
    pepper_pixels = int(pepper_prob * total_pixels)

    # Add salt (white) pixels
    for _ in range(salt_pixels):
        i = random.randint(0, image.shape[0] - 1)
        j = random.randint(0, image.shape[1] - 1)
        noisy[i, j] = 255

    # Add pepper (black) pixels
    for _ in range(pepper_pixels):
        i = random.randint(0, image.shape[0] - 1)
        j = random.randint(0, image.shape[1] - 1)
        noisy[i, j] = 0

    return noisy

In [4]:
def add_speckle_noise(image, mean=0, var=0.2):
    """
    Adds speckle noise to an image.
    
    Parameters:
        image (ndarray): Input image (grayscale or RGB).
        mean (float): Mean of the Gaussian noise.
        var (float): Variance of the Gaussian noise.
    
    Returns:
        ndarray: Noisy image with speckle noise added.
    """
    # Generate Gaussian noise
    noise = np.random.normal(mean, np.sqrt(var), image.shape)
    
    # Add the noise (multiplicative noise)
    noisy_image = image + image * noise
    
    # Clip values to ensure valid pixel intensity range [0, 255]
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    
    return noisy_image

In [5]:
def rotate_circular_region(image, center=None, angle=None, radius=None):
    """
    Rotates a circular region of an image with random parameters if not provided.
    
    Parameters:
        image (ndarray): Input image.
        center (tuple, optional): Center of the circular region. Random if None.
        angle (float, optional): Angle of rotation. Random if None.
        radius (int, optional): Radius of the circular region. Random if None.
    
    Returns:
        ndarray: Image with the rotated circular region.
    """
    height, width = image.shape[:2]

    # Generate random parameters if not provided
    if center is None:
        center = (
            random.randint(75, width - 75),  # Random x-coordinate
            random.randint(75, height - 75)  # Random y-coordinate
        )
    if angle is None:
        angle = random.choice([90, 135, 180])  # Random angle
    if radius is None:
        radius = (100)  # Random radius

    # Create a mask for the circular region
    mask = np.zeros_like(image, dtype=np.uint8)
    cv2.circle(mask, center, radius, (255, 255, 255), -1)

    # Extract the circular region using the mask
    circular_roi = cv2.bitwise_and(image, image, mask=mask)

    # Create a bounding box around the circle
    x, y = center
    x1, y1 = x - radius, y - radius
    x2, y2 = x + radius, y + radius
    circular_roi = circular_roi[y1:y2, x1:x2]

    # Ensure the ROI is square (width and height of the circle bounding box)
    if circular_roi.shape[0] != circular_roi.shape[1]:
        size = max(circular_roi.shape[:2])
        circular_roi = cv2.resize(circular_roi, (size, size))

    # Rotate the circular ROI
    matrix = cv2.getRotationMatrix2D((radius, radius), angle, 1.0)
    rotated_roi = cv2.warpAffine(circular_roi, matrix, (2 * radius, 2 * radius))

    # Create a mask for the rotated circle
    rotated_mask = np.zeros((2 * radius, 2 * radius), dtype=np.uint8)
    cv2.circle(rotated_mask, (radius, radius), radius, 255, -1)

    # Extract the rotated circle using the rotated mask
    rotated_circle = cv2.bitwise_and(rotated_roi, rotated_roi, mask=rotated_mask)

    # Place the rotated circle back into the original image
    result = image.copy()
    for i in range(-radius, radius):
        for j in range(-radius, radius):
            if rotated_mask[radius + j, radius + i]:
                xi, yi = x + i, y + j
                if 0 <= xi < result.shape[1] and 0 <= yi < result.shape[0]:
                    result[yi, xi] = rotated_circle[radius + j, radius + i]

    return result

In [6]:
def add_white_smudge(image, num_smudges=None, max_radius=30):
    """
    Adds random white smudge-like blur spots to the fingerprint image.

    Parameters:
        image: Input image (numpy array).
        num_smudges: Number of white smudges to add.
        max_radius: Maximum radius of the smudge region.

    Returns:
        Image with random white smudges.
    """
    # Generate random parameters if not provided
    if num_smudges is None:
        num_smudges = random.choice([4,6])
    # Ensure the image is grayscale
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    smudged_image = image.copy()
    height, width = image.shape
    
    for _ in range(num_smudges):
        # Random center and radius
        center_x = random.randint(0, width - 1)
        center_y = random.randint(0, height - 1)
        radius = random.randint(10, max_radius)

        # Create a white "smudge" on a black background
        smudge = np.zeros((height, width), dtype=np.uint8)
        cv2.circle(smudge, (center_x, center_y), radius, 255, -1)

        # Apply Gaussian blur to the smudge
        smudge = cv2.GaussianBlur(smudge, (radius // 2 * 2 + 1, radius // 2 * 2 + 1), 0)

        # Blend the smudge into the image (lighter smudges)
        # Chinh do sang bang cach tang giam he so thu 4
        smudged_image = cv2.addWeighted(smudged_image, 1, smudge, .7, 0)
    
    return smudged_image

In [7]:
def add_black_smudge(image, num_smudges=None, max_radius=30):
    """
    Adds random white smudge-like blur spots to the fingerprint image.

    Parameters:
        image: Input image (numpy array).
        num_smudges: Number of white smudges to add.
        max_radius: Maximum radius of the smudge region.

    Returns:
        Image with random white smudges.
    """
    # Generate random parameters if not provided
    if num_smudges is None:
        num_smudges = random.choice([4,6])
    # Ensure the image is grayscale
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    smudged_image = image.copy()
    height, width = image.shape
    
    for _ in range(num_smudges):
        # Random center and radius
        center_x = random.randint(0, width - 1)
        center_y = random.randint(0, height - 1)
        radius = random.randint(10, max_radius)

        # Create a white "smudge" on a black background
        smudge = np.zeros((height, width), dtype=np.uint8)
        cv2.circle(smudge, (center_x, center_y), radius, 255, -1)

        # Apply Gaussian blur to the smudge
        smudge = cv2.GaussianBlur(smudge, (radius // 2 * 2 + 1, radius // 2 * 2 + 1), 0)

        # Blend the smudge into the image (lighter smudges)
        # Chinh do sang bang cach tang giam he so thu 4
        smudged_image = cv2.addWeighted(smudged_image, 1, smudge, -.7, 0)
    
    return smudged_image

In [8]:
def add_both_smudge(image):
    result = add_black_smudge(image)
    result = add_white_smudge(result)
    return result

In [9]:
def add_random_lines(image, line_count=10, thickness=10):
    noisy = np.copy(image)
    h, w = image.shape
    for _ in range(line_count):
        pt1 = (random.randint(0, w), random.randint(0, h))
        pt2 = (random.randint(0, w), random.randint(0, h))
        color = random.randint(0, 255)
        cv2.line(noisy, pt1, pt2, color, thickness)
    return noisy

#line_image = add_random_lines(image, line_count=15, thickness=1)

In [10]:
def elastic_transform_partial(image, alpha = 100, sigma = 5, roi=None, random_state=None):
    """
    Applies an elastic deformation to a region of interest (ROI) in an image.

    Parameters:
        image (ndarray): Input image (grayscale or RGB).
        alpha (float): Scaling factor for the deformation intensity.
        sigma (float): Standard deviation for the Gaussian filter (smoothness of deformation).
        roi (tuple): Region of interest as (x, y, width, height). If None, apply to the whole image.
        random_state (int, optional): Seed for reproducibility.

    Returns:
        ndarray: Image with elastic deformation applied to the ROI.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)
    
    # If no ROI is provided, use the whole image
    if roi is None:
        x = 100
        y = 100
        w = random.randint(200, 300)
        h = random.randint(200, 300)
        roi = (x, y, w, h)

    # Extract ROI coordinates
    x, y, w, h = roi
    roi_image = image[y:y+h, x:x+w]

    # Image dimensions of ROI
    shape = roi_image.shape[:2]

    # Generate random displacement fields
    dx = random_state.rand(*shape) * 2 - 1  # Random values in [-1, 1]
    dy = random_state.rand(*shape) * 2 - 1  # Random values in [-1, 1]

    # Apply Gaussian filter to smooth the displacements
    dx = cv2.GaussianBlur(dx, (0, 0), sigma) * alpha
    dy = cv2.GaussianBlur(dy, (0, 0), sigma) * alpha

    # Generate meshgrid of coordinates
    x_coords, y_coords = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
    map_x = (x_coords + dx).astype(np.float32)
    map_y = (y_coords + dy).astype(np.float32)

    # Apply the elastic transformation using remapping
    transformed_roi = cv2.remap(
        roi_image,
        map_x,
        map_y,
        interpolation=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_REFLECT_101
    )

    # Replace the transformed ROI in the original image
    result = image.copy()
    result[y:y+h, x:x+w] = transformed_roi

    return result

In [11]:
def rotate_and_crop(image, angle=None):
    """
    Rotates an image around its center and crops the output to match the original dimensions.

    Parameters:
        image (numpy array): Input image.
        angle (float): Angle to rotate the image (in degrees, counter-clockwise).

    Returns:
        Rotated and cropped image (numpy array).
    """
    # Get the dimensions of the original image
    (height, width) = image.shape[:2]

    # Get the center of the image
    center = (width // 2, height // 2)

    if angle is None:
        counterclockwise = random.randint(10, 30)
        clockwise = random.randint(-30, -10)
        angle = random.choice([clockwise, counterclockwise])

    # Compute the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Perform the rotation with white border
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height),
                                   borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))

    # Crop the rotated image to match the original dimensions
    # Ensure the cropping area is valid
    x_start = (rotated_image.shape[1] - width) // 2
    y_start = (rotated_image.shape[0] - height) // 2

    cropped_image = rotated_image[y_start:y_start + height, x_start:x_start + width]
    
    return cropped_image

In [12]:
def blur_then_smudge(image):
    result = cv2.GaussianBlur(image, (15, 15), 0)
    result = add_random_lines(result)
    return result

In [13]:
def display_images(original, noisy, title="Noisy Image"):
    """Displays the original and noisy images side by side."""
    combined = np.hstack((original, noisy))
    cv2.imshow(title, combined)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [20]:
def augmentation(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    total_images = len(image_files)
    if total_images < 8000:
            print("Warning: Less than 4000 images found. Processing all available images.")
    else:
        image_files = image_files[:8000]  # Limit to 4000 images
    
    for idx in range(8000):
        start = idx * 8000
        end = start + 8000
        current_batch = image_files[start:end]

        for file in current_batch:
            input_path = os.path.join(input_folder, file)
            image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

            if image is None:
                print(f"Error reading {file}. Skipping.")
                continue

            # Apply augmentation        
            aug_image = rotate_and_crop(image)

            # Save the augmented image
            output_path = os.path.join(output_folder, f"rotated_{file}")
            cv2.imwrite(output_path, aug_image)

if __name__ == "__main__":
    input_folder = "C:\\Users\\84965\\Documents\\Fingerprint_Recognition_GROUP8\\augmented2"
    output_folder = "C:\\Users\\84965\\Documents\\Fingerprint_Recognition_GROUP8\\augmented1"
    augmentation(input_folder, output_folder)



In [14]:
def process_images_in_batches(input_folder, output_folder):
    """Processes 4000 images in 800-image batches with different augmentations."""
    os.makedirs(output_folder, exist_ok=True)
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    total_images = len(image_files)

    if total_images < 4000:
        print("Warning: Less than 16000 images found. Processing all available images.")
    else:
        image_files = image_files[:4000]  # Limit to 16000 images

    # Split into 5 portions of 800 images
    batch_size = 800
    augmentation_methods = [
        lambda image: cv2.GaussianBlur(image, (15, 15), 0),
        elastic_transform_partial,
        add_random_lines,
        add_both_smudge,
        blur_then_smudge,
    ]

    for idx, method in enumerate(augmentation_methods):
        start = idx * batch_size
        end = start + batch_size
        current_batch = image_files[start:end]

        print(f"Processing batch {idx + 1} with {method.__name__ if method != 'random' else 'Random Augmentations'}")

        for file in current_batch:
            input_path = os.path.join(input_folder, file)
            image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

            if image is None:
                print(f"Error reading {file}. Skipping.")
                continue

            # Apply augmentation        
            aug_image = method(image)

            # Save the augmented image
            output_path = os.path.join(output_folder, f"batch{idx+1}_{file}")
            cv2.imwrite(output_path, aug_image)

    print("Processing complete.")

def random_augmentation(image):
    """Applies 1-2 random augmentations."""
    methods = [
        #lambda img: add_gaussian_noise(img),
        lambda img: add_salt_and_pepper_noise(img),
        #lambda img: add_speckle_noise(img),
        lambda img: rotate_circular_region(img),
        lambda img: add_white_smudge(img)
    ]
    aug_image = image.copy()
    for _ in range(2):
        method = random.choice(methods)
        aug_image = method(aug_image)
    return aug_image

if __name__ == "__main__":
    input_folder = "C:\\Users\\ADMIN\\FingerprintClassification\\model\\fingerprint"
    output_folder = "C:\\Users\\ADMIN\\FingerprintClassification\\model\\fingerprint_noise"
    process_images_in_batches(input_folder, output_folder)


Processing batch 1 with <lambda>
Processing batch 2 with elastic_transform_partial
Processing batch 3 with add_random_lines
Processing batch 4 with add_both_smudge
Processing batch 5 with blur_then_smudge
Processing complete.


In [108]:
# Load a fingerprint image
img = cv2.imread('C:\\Users\\84965\\Documents\\Fingerprint_Recognition_GROUP8\\model\\anhhvantay\\A_f0823_06.png', cv2.IMREAD_GRAYSCALE)
#blur
#white+black smudge
#

#a = cv2.GaussianBlur(image, (15, 15), 0)
#a = elastic_transform_partial(img, 100, 5,)
#a = flipped_image = cv2.flip(img, 1)

# Show the result
cv2.imwrite('C:\\Users\\84965\\Documents\\Fingerprint_Recognition_GROUP8\\model\\a.png', a)
cv2.waitKey(0)
cv2.destroyAllWindows()

noise:
- blur
- smudge
- random lines
- lastic_transform_partial

1900 636769 
augmentation:
- rotate 
- flip